In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner-with-dates")

model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner-with-dates")

##### Process text sample (from wikipedia)

C:\Users\rrajab\Anaconda3\envs\envNER\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import csv
from transformers import pipeline
def isNaN(string):
    return string != string
extractionEntity = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")
df = pd.read_csv('data.csv')
vraiPositif=0
counter = 0
extrait = 0
wordList = ["le", "un", "en"]
with open('result.csv', 'w', newline='', encoding="utf8") as file:
    writer = csv.writer(file, delimiter =';')
    writer.writerow(["index", "text", "extraite", "score", "annotée"])
    for i in df.index:
        if not isNaN(df['texte'][i]):
            entities = extractionEntity(df['texte'][i])
            for entity in entities :
                    if (entity['entity_group'] == "DATE") :
                        extrait+=1
                        if any(word in entity['word'] for word in wordList): 
                            counter+=1
                        #print(i, "text = ",df['texte'][i], "|||| extraite =", entity['word'] ,"|||| score = ", entity['score'] ,"|||| annotée =", df['dateNes'][i])
                        writer.writerow([i,df['texte'][i], entity['word'], entity['score'], df['dateNes'][i]])
                    
    file.close()

with open('result.csv', 'r', encoding="utf8") as file:
    
    csv_reader = csv.reader(file, delimiter=';')
    uniqueIds = set()

    for row in csv_reader:
        uniqueIds.add(row[0])

datafficher = len(uniqueIds)


originalFileDates = 490
orginalfiledata=1541
fauxPositif = 3
vraiPositif = datafficher - fauxPositif
fauxNegatif = originalFileDates - vraiPositif
vraiNegatif = orginalfiledata-originalFileDates-fauxPositif
precision = vraiPositif/(vraiPositif+fauxPositif)
rappel =  vraiPositif/(vraiPositif+fauxNegatif)
f1score=(2*precision*rappel)/(precision+rappel)
Accuracy=(vraiNegatif +vraiPositif)/(vraiNegatif+fauxPositif+vraiPositif+fauxNegatif)


with open('stat.csv', 'w', newline='', encoding="utf8") as file:
    writer = csv.writer(file, delimiter =';')
    writer.writerow(["originalFileDates","fauxNegatif", "vraiPositif", "fauxPositif","vraiNegatif", "precision","rappel", "f1score","Accuracy"])
    writer.writerow([originalFileDates,fauxNegatif, vraiPositif, fauxPositif, vraiNegatif, precision, rappel, f1score, Accuracy])
    file.close()

repetation = extrait - datafficher

print("Number data in orginal file  = " , orginalfiledata)
print("Number des dates existe in original file data= " , originalFileDates)
print("Number de data extrait = " ,extrait )
print("Number des data afficher sans repetation = " , datafficher)
print("Number des dates afficher sans repetation = " , vraiPositif)
print ("Number des dates extratit avec special caracters = ", counter)
print("Nombre de repetation existe dans l'extrait = " , repetation)



Number data in orginal file  =  1541
Number des dates existe in original file data=  490
Number de data extrait =  501
Number des data afficher sans repetation =  493
Number des dates afficher sans repetation =  490
Number des dates extratit avec special caracters =  359
Nombre de repetation existe dans l'extrait =  8
